# Modelo de la turbina

## Bibliotecas

In [ ]:
#Aprendizaje por refuerzo
from gym import Env
from gym.spaces import Discrete, Box
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

#Red neuronal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

#Arrays y operaciones aritméticas
import numpy as np
import random
import math

#Aplicación web
from werkzeug.wrappers import Request, Response
from werkzeug.serving import run_simple
from flask import Flask, render_template, request, redirect, url_for, flash
from wtforms import Form, FloatField, validators

#Gráficos e impresión por pantalla
import io
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import base64

## Entorno

In [ ]:
class WindmillEnv(Env):
    def __init__(self, pRef):
        
        #Número de acciones
        self.action_space = Discrete(7)
        
        #Espacio de observación
        self.observation_space = Box(low=np.array([5]), high=np.array([14]))
        
        #Tiempo de entrenamiento
        self.training_length = 1000
        
        #Parámetros de la turbina
        #Variables estáticas
        self.wind_density = 1.225
        self.radious = 2
        self.wind = 10.0
        self.powerRef = pRef
        
        #Variables dinámicas
        self.angle = random.uniform(5.0, 14.0)
        self.power_eficiency = (-0.0422)*self.angle + 0.5911
        self.genPowerEuler = 0.5*self.wind_density*math.pi*pow(self.radious, 2)*pow(self.wind, 3)*self.power_eficiency
        self.error = abs(self.powerRef - self.genPowerEuler)
    
    
    def step(self, action):
        
        powerRefCheck = env.powerRef
        
        #Guardamos el error del paso anterior
        last_error = self.error
        
        #Reducimos el tiempo de entrenamiento en 1 segundo
        self.training_length -= 1
        
        #Aplicamos la acción
        self.angle += (action/10.0) - 0.01
         
        #Linealizamos el modelo de la turbina
        for t in range(1, 151):
            self.power_eficiency = (-0.0422)*self.angle + 0.5911
            self.genPowerEuler += ((0.5*self.wind_density*math.pi*pow(self.radious, 2)*pow(self.wind, 3)
                                    *self.power_eficiency)/5 - self.genPowerEuler/5)*0.5
        
        #Calculamos el error actual
        self.error = abs(powerRefCheck - self.genPowerEuler)
        
        
        #Calculamos la recompensa
        if self.error < last_error:
            reward = 1 - (self.error/10)
        if self.error > last_error:
            reward = -100 - (self.error/10)
        else:
            reward = -50 - (self.error/10)
            
        #Comprobamos si el tiempo de entrenamiento ha llegado a 0
        if self.training_length <= 0:
            done = True
        else:
            done = False
                
        #Info
        info = {}
        
        return self.angle, reward, done, info
    
    #Función para resetear los parámetros dinámicos
    def reset(self):
        
        self.angle = random.uniform(5, 14)
        self.power_eficiency = (-0.0422)*self.angle + 0.5911
        self.genPowerEuler = 0.5*self.wind_density*math.pi*pow(self.radious, 2)*pow(self.wind, 3)*self.power_eficiency
        self.error = abs(self.powerRef - self.genPowerEuler)
        
        self.training_length = 1000
        
        return self.angle
    
    
    #GETTERS Y SETTERS
    #Potencia de referencia
    @property
    def powerRefMethod(self):
        return self.powerRef
    
    @powerRefMethod.setter
    def powerRefMethod(self, powerRefv):
        self.powerRef = powerRefv
    
    
    #Potencia generada
    @property
    def genPowerEulerMethod(self):
        return self.genPowerEuler
    
    @genPowerEulerMethod.setter
    def genPowerEulerMethod(self, genPowerEulerv):
        self.genPowerEuler = genPowerEulerv
    
    
    #Ángulo
    @property
    def angleMethod(self):
        return self.angle
    
    @angleMethod.setter
    def angleMethod(self, anglev):
        self.angle = anglev
    
    
    #Tiempo de entrenamiento
    @property
    def training_lengthMethod(self):
        return self.training_length
    
    @training_lengthMethod.setter
    def training_lengthMethod(self, training_lengthv):
        self.training_length = training_lengthv
        
    

In [ ]:
#Instanciamos el entorno en una variable
env = WindmillEnv(1000)
states = env.observation_space.shape
actions = env.action_space.n

## Red neuronal

In [ ]:
#Función para crer el modelo de la red neuronal
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape = states))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)

## Agente DQN

In [ ]:
#Función para crear el agente
def build_agent(model, actions):
    memory = SequentialMemory(limit=30000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=BoltzmannQPolicy(), nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [ ]:
lastExecution_powerR = 0

## Ejecución de la interfaz

In [ ]:
app = Flask(__name__)

class InputForm(Form):
    #Variable para recoger el valor de la potencia introducida
    r = FloatField(validators=[validators.InputRequired()])

@app.route("/", methods=["POST", "GET"])
def mainFunction():
    
    form = InputForm(request.form)
    
    if request.method == "POST":
        #Comprobación de seguridad
        if (isinstance(form.r.data, (int, float)) is not True) or (form.r.data > 2950.0) or (form.r.data < 100.0):
            flash('El valor introducido debe ser un número entre 100 y 2900.')
            return render_template("RL.html", form=form)
        
        global lastExecution_powerR
        global dqn
        
        powerR = form.r.data
        env.powerRefMethod = powerR
        
        #Comprobación para no entrenar dos veces seguidas el modelo para la misma potencia
        if (request.form['submit_button'] == 'Entrenado') and (powerR != lastExecution_powerR):
            #Entrenamiento del agente
            dqn = build_agent(model, actions)
            dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
            dqn.fit(env, nb_steps=30000, visualize=False, verbose=1)   
            lastExecution_powerR = powerR
        
        #Reseteamos las variables del entorno
        obs = env.reset()
        done = False
        score = 0
        
        powerArray = []
        anglesArray = []
        initPower = env.genPowerEulerMethod
        #Se ha aumentado el tiempo de entrenamiento para dar mas margen al modelo
        env.training_lengthMethod = 2000
        powerArray.append(env.genPowerEulerMethod)
        anglesArray.append(env.angleMethod)
    
        while not done:
            #Dependiendo del botón pulsado, las acciones son aleatorias o las toma el modelo entrenado
            if request.form['submit_button'] == 'Sin entrenar':
                action = env.action_space.sample()
            else:
                action = dqn.forward(obs)
                    
            obs, reward, done, info = env.step(action)
            score += reward
        
            powerArray.append(env.genPowerEulerMethod)
            anglesArray.append(env.angleMethod)
            
            #Condición de parada para un error menor a 2Kw
            if abs(env.powerRefMethod - env.genPowerEulerMethod) < 2.0:
                break
        
        #Gráficas con los resultados de la ejecución
        figure = Figure()
        figure.set_size_inches(18.5, 10.5)

        plt1 = figure.add_subplot(1,2,1)
        plt1.set_title("Potencia generada")
        plt1.axhline(y=powerR, color='r', linestyle='-')
        plt1.set_xlabel("Pasos")
        plt1.set_ylabel("Potencia")
        plt1.plot(powerArray, 'b')
        
        plt2 = figure.add_subplot(1,2,2)
        plt2.set_title("Ángulo del aspa")
        plt2.set_xlabel("Pasos")
        plt2.set_ylabel("Ángulo")
        plt2.plot(anglesArray, 'g')
        
        output = io.BytesIO()
        figure.savefig(output, format='png')
        plotData = base64.b64encode(output.getbuffer()).decode("ascii")
        
        return render_template("RL.html", form=form, plotImg=plotData, finalPower=env.genPowerEulerMethod, initPower=initPower, finalAngle=env.angleMethod)
    else:
        
        return render_template("RL.html", form=form)

if __name__ == '__main__':
    app.secret_key="anystringhere"
    run_simple('localhost', 8000, app)
    
lastExecution_powerR = 0